### Community detection in directed graphs by applying spectral methods to the modularity matrix

See Leight and Newman, 2008, "Community Structure in Directed Networks". See also the undirected modularity file, which has a walkthrough of how similar code is structured. 

In [6]:
import numpy as np
import matplotlib.pyplot as plt

In [7]:
def get_split(B, m):
    ews, evs = np.linalg.eig(B + B.T)
    s = np.sign(evs[:, 0])
    dQ = np.dot(s, np.dot(B + B.T, s)) / (4 * m)
    return s, dQ

In [8]:
def get_Bg(B, idx):
    n = np.shape(idx)[0]
    Bg = np.copy(B)

    for i in range(n):
        Bg[i, i] -= (np.sum(B[i, idx]) + np.sum(B[idx, i])) / 2

    return Bg[idx, :][:, idx]

In [9]:
def modularity_parition(adj, tol = 0.001):
    n = np.shape(adj)[0]
    m = np.sum(adj) # total number of edges

    labels = np.zeros(n) # track vertex labels
    Q = 0 # modularity
    last_comm = 0 # previously labeled community
    queue = [0]

    deg_in = np.zeros(n)
    deg_out = np.zeros(n)
    for i in range(n):
        deg_in[i] = np.sum(adj[:, i])
        deg_out[i] = np.sum(adj[i, :])
    
    B = adj - np.outer(deg_in, deg_out) / m

    while(len(queue) > 0):

        comm = queue.pop(0)
        idx = labels == comm
        
        Bg = get_Bg(B, idx)
        s, dQ = get_split(Bg, m)

        # help indexing
        if dQ > 0.001:
            labels[idx] += last_comm + (s + 3)/2
            queue.append(last_comm + 1)
            queue.append(last_comm + 2)

            last_comm = last_comm + 2
            Q = Q + dQ

        return s, Q
    